In [1]:
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

In [2]:
(training_features, training_labels) , (testing_features, testing_labels) = datasets.cifar100.load_data(label_mode='fine')

In [3]:
training_features_scaled, testing_features_scaled = training_features / 255.0, testing_features / 255.0
#Original training features and testing features are integer arrays, scaling and converting to float

In [4]:
LABEL_NAMES = ['apple','aquarium_fish','baby','bear','beaver','bed','bee','beetle','bicycle','bottle','bowl','boy','bridge',
'bus','butterfly','camel','can','castle','caterpillar','cattle','chair','chimpanzee','clock','cloud','cockroach','couch',
'crab','crocodile','cup','dinosaur','dolphin','elephant','flatfish','forest','fox','girl','hamster','house','kangaroo',
'keyboard','lamp','lawn_mower','leopard','lion','lizard','lobster','man','maple_tree','motorcycle','mountain','mouse',
'mushroom','oak_tree','orange','orchid','otter','palm_tree','pear','pickup_truck','pine_tree','plain','plate','poppy',
'porcupine','possum','rabbit','raccoon','ray','road','rocket','rose','sea','seal','shark','shrew','skunk','skyscraper',
'snail','snake','spider','squirrel','streetcar','sunflower','sweet_pepper','table','tank','telephone','television','tiger',
'tractor','train','trout','tulip','turtle','wardrobe','whale','willow_tree','wolf','woman','worm']

In [ ]:
training_features_scaled.shape

In [5]:
model = models.Sequential()

#Conv layer
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(64, (3, 3), activation='relu'))

#Dense layer
model.add(layers.Flatten()) #need to make the layer linear to connect to Dense layers
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(100))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 10816)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               5538304   
_________________________________________________________________
dense_1 (Dense)              (None, 100)               51300     
Total params: 5,628,324
Trainable params: 5,628,324
Non-trainable params: 0
______________________________________________

##### Seems like ReLU preferred for images analysis? Also cannot use too many layers or number per layer..

In [6]:
model.compile(optimizer=keras.optimizers.Adam(),loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True)
             , metrics=[keras.metrics.SparseCategoricalAccuracy()])
#need to use SparseCategoricalAccuracyinstead of other types of accuracies..

In [ ]:
test = model.fit(training_features_scaled,training_labels,epochs=1,validation_split=0.1)

In [ ]:
predictions = model.predict(testing_features_scaled)

In [ ]:
def prediction(index):
    plt.imshow( testing_features[index] )
    print("The actual picture is a " +LABEL_NAMES[ int( testing_labels[index] ) ] )
    print("The predicted picture is a " +LABEL_NAMES[predictions[index].argmax()])
    
prediction(4862)

### Using pretrained models

In [ ]:
base_model = keras.applications.VGG16(input_shape=[32, 32, 3],include_top=False,weights='imagenet')


In [ ]:
base_model.summary()
base_model.trainable = False

In [ ]:
global_average_layer = keras.layers.GlobalAveragePooling2D()

In [ ]:
prediction_layer = keras.layers.Dense(100)

In [ ]:
model2 = keras.Sequential([base_model,global_average_layer,prediction_layer])

In [ ]:
model2.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model2.summary()

In [ ]:
letsgo = model2.fit(training_features_scaled,training_labels,epochs=1,validation_split=0.1)

Epoch 1/10
1407/1407 [==============================] - 16s 10ms/step - loss: 3.4285 - accuracy: 0.2109 - val_loss: 3.0777 - val_accuracy: 0.2640
Epoch 2/10
1407/1407 [==============================] - 13s 9ms/step - loss: 2.8858 - accuracy: 0.2970 - val_loss: 2.8968 - val_accuracy: 0.2988
Epoch 3/10
1407/1407 [==============================] - 13s 9ms/step - loss: 2.7181 - accuracy: 0.3304 - val_loss: 2.8238 - val_accuracy: 0.3084
Epoch 4/10
1407/1407 [==============================] - 13s 9ms/step - loss: 2.6139 - accuracy: 0.3488 - val_loss: 2.7588 - val_accuracy: 0.3226
Epoch 5/10
1407/1407 [==============================] - 13s 9ms/step - loss: 2.5398 - accuracy: 0.3632 - val_loss: 2.7394 - val_accuracy: 0.3294
Epoch 6/10
1407/1407 [==============================] - 13s 9ms/step - loss: 2.4810 - accuracy: 0.3748 - val_loss: 2.7017 - val_accuracy: 0.3384
Epoch 7/10
1407/1407 [==============================] - 13s 9ms/step - loss: 2.4334 - accuracy: 0.3848 - val_loss: 2.6841 - val_accuracy: 0.3406
Epoch 8/10
1407/1407 [==============================] - 13s 9ms/step - loss: 2.3923 - accuracy: 0.3914 - val_loss: 2.6799 - val_accuracy: 0.3388
Epoch 9/10
1407/1407 [==============================] - 14s 10ms/step - loss: 2.3581 - accuracy: 0.3989 - val_loss: 2.6522 - val_accuracy: 0.3484
Epoch 10/10
1407/1407 [==============================] - 13s 9ms/step - loss: 2.3270 - accuracy: 0.4051 - val_loss: 2.6533 - val_accuracy: 0.3486

##### Low accuracy compared to my simple model built from scratch.., perhaps too many conv / pooling layers for simple 32x32 pixel pictures? Maybe if picture is higher resolution then accuracy would be higher using pretrained models with many layers like the VGG16